## Keras
Notes from Lukas Biewald's [Crowdflower Machine Learning class](https://github.com/lukas/ml-class)

### Perceptron Model

In [1]:
#digits.py
from keras.datasets import mnist
import numpy as np

(X_train, y_train), (X_test, y_test) = mnist.load_data()

idx = 153
digit = X_train[idx]

str= ""
for i in range(digit.shape[0]):
    for j in range(digit.shape[1]):
        if digit[i][j] == 0:
            str += ' '
        elif digit[i][j] < 128:
            str += '.'
        else:
            str += 'X'
    str += "\n"

print(str)

Using TensorFlow backend.


11444224/11490434 [============================>.] - ETA: 0s                            
                            
                            
                            
                            
                            
                            
            .XXXXX..        
           .XXXX.XXX        
          .XX.    X.        
          XX.   .XX.        
         .XX.  ..XX         
         .X.  ..XX.         
         XXX..XXXX.         
         .XXXXXXX..         
          .X...XX           
              XX.           
             .XX            
             XX.            
            .XX.            
            XXX             
           .XX.             
          .XX.              
          XX.               
         .X.                
        .X..                
        XX                  
                            

Label:, y_train[idx]


In [ ]:
print("Label:" , y_train[idx])
#=> Label: 9

Train a perceptron using the flattened digit arrays:

In [8]:
#keras-scikit-learn.py
from keras.utils import np_utils
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score

X_train = [x.flatten() for x in X_train]
perceptron = Perceptron()
scores = cross_val_score(perceptron, X_train, y_train, cv=10)
scores.mean()
#=> 0.85830929206963147

0.85830929206963147

### Sequential Model

Normalize digit pixel intensities by dividing by the max value of 255:

In [66]:
#perceptron-checkpoint.py
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

Transform the outputs using [one-hot encoding](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/). This a pre-processing step needed to feed categorical data to many scikit-learn estimators.

`to_categorical` converts a class vector of integers to binary class matrix.

In [67]:
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]

y_test = np_utils.to_categorical(y_test)

Create a [Sequential](https://keras.io/getting-started/sequential-model-guide/) model by passing a list of layers and specifying an input shape. `Dense` is a fully-connected layer with the specified number of units. `Compile` configures the learning process and accepts an [optimizer](https://keras.io/optimizers/), a [loss](https://keras.io/losses/) function and a list of metrics. For classification problems, set metrics as ['accuracy'].

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
model=Sequential()
model.add(Flatten(input_shape=(img_width,img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('model', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[checkpoint])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.4689 - acc: 0.8790 - val_loss: 0.3069 - val_acc: 0.9146
Epoch 2/10
60000/60000 [==============================] - 2s - loss: 0.3037 - acc: 0.9147 - val_loss: 0.2829 - val_acc: 0.9208
Epoch 3/10
60000/60000 [==============================] - 2s - loss: 0.2834 - acc: 0.9210 - val_loss: 0.2727 - val_acc: 0.9243
Epoch 4/10
60000/60000 [==============================] - 2s - loss: 0.2727 - acc: 0.9238 - val_loss: 0.2734 - val_acc: 0.9223
Epoch 5/10
60000/60000 [==============================] - 2s - loss: 0.2660 - acc: 0.9265 - val_loss: 0.2731 - val_acc: 0.9236
Epoch 6/10
60000/60000 [==============================] - 2s - loss: 0.2615 - acc: 0.9273 - val_loss: 0.2676 - val_acc: 0.9248
Epoch 7/10
60000/60000 [==============================] - 2s - loss: 0.2578 - acc: 0.9285 - val_loss: 0.2666 - val_acc: 0.9272
Epoch 8/10
60000/60000 [==============================] - 2s 

Evaluate the final model and save.

In [70]:
scores = model.evaluate(X_test, y_test, verbose=0)
print(scores)
model.save("sequential.h5")

[0.26729143058955668, 0.92669999999999997]


Load the model and display the weights.

In [76]:
from keras.models import load_model

saved_model = load_model("sequential.h5")
print(saved_model.layers[1].get_weights())

[array([[-0.08640513,  0.05191677, -0.0533197 , ...,  0.04133975,
        -0.02808482, -0.05308444],
       [ 0.020721  , -0.04056662,  0.02190006, ..., -0.07330699,
         0.07682715, -0.01448923],
       [ 0.08531713, -0.03980877,  0.08151612, ...,  0.01148049,
        -0.03928037,  0.01881983],
       ..., 
       [ 0.00479493,  0.07352971, -0.04358268, ..., -0.00106092,
         0.0591091 ,  0.03954609],
       [-0.00603664,  0.07396612, -0.03519871, ..., -0.07693817,
         0.03887091, -0.01640342],
       [ 0.085405  , -0.05702055,  0.0396649 , ..., -0.03235378,
         0.08464644, -0.04457386]], dtype=float32), array([-0.53456676,  0.65486735,  0.14239714, -0.43005994,  0.14697094,
        1.39619493, -0.16160907,  0.73810667, -1.49492216, -0.30620834], dtype=float32)]


### Logging

Save history of weights and models using [Wandb](https://github.com/wandb/client) callback and visualize log data using [TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard):

In [85]:
import json
import wandb

from keras.callbacks import TensorBoard
#from wandb.wandb_keras import WandbKerasCallback

#run = wandb.init()
#config = run.config
#TODO:
#1. Use config epochs and batch_size as model parameters
#2. Add WandKerasCallback to callbacks after Wandb invite is activated

tensorboard = TensorBoard(log_dir="logs")

history = model.fit(X_train, y_train, epochs=10,
        batch_size=10, validation_data=(X_test, y_test),
        callbacks=[tensorboard])

scores = model.evaluate(X_test, y_test, verbose=0)

with open('metrics.json', 'w') as outfile:
    json.dump(scores, outfile)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s - loss: 0.2575 - acc: 0.9280 - val_loss: 0.2680 - val_acc: 0.9265
Epoch 2/10
60000/60000 [==============================] - 8s - loss: 0.2531 - acc: 0.9301 - val_loss: 0.2711 - val_acc: 0.9259
Epoch 3/10
60000/60000 [==============================] - 8s - loss: 0.2511 - acc: 0.9307 - val_loss: 0.2737 - val_acc: 0.9250
Epoch 4/10
60000/60000 [==============================] - 8s - loss: 0.2497 - acc: 0.9310 - val_loss: 0.2746 - val_acc: 0.9270
Epoch 5/10
60000/60000 [==============================] - 8s - loss: 0.2478 - acc: 0.9319 - val_loss: 0.2732 - val_acc: 0.9260
Epoch 6/10
60000/60000 [==============================] - 8s - loss: 0.2468 - acc: 0.9315 - val_loss: 0.2803 - val_acc: 0.9238
Epoch 7/10
60000/60000 [==============================] - 9s - loss: 0.2464 - acc: 0.9321 - val_loss: 0.2812 - val_acc: 0.9260
Epoch 8/10
60000/60000 [==============================] - 8s 

### Visualization

Launch TensorBoard `tensorboard --logdir=/path/to/dir` then open `localhost:6006` to on your browser to view the graphs.

<img src="tensorboard-sequential-metrics.png" width=600 height=300>